In [1]:
# Одно из классических предписаний для классов — у каждого из множества объектов есть некоторые меняющиеся состояния. 

# Вернёмся к примеру: есть база клиентов с основной информацией; в реальном времени нам приходит информация о покупках. 
# Запустим промокампанию, чтобы поощрить старых клиентов, которые сделали у нас много заказов, и выдать им скидку:

class Client():  
    # Базовые данные  
    def __init__(self, email, order_num, registration_year):  
        self.email = email  
        self.order_num = order_num  
        self.registration_year = registration_year  
        self.discount = 0  
          
    # Оформление заказа  
    def make_order(self, price):  
        self.update_discount()  
        self.order_num += 1  
        # Здесь было бы оформление заказа, но мы просто выведем его цену  
        discounted_price = price * (1 - self.discount)   
        print(f"Order price for {self.email} is {discounted_price}")  
              
    # Назначение скидки  
    def update_discount(self):   
        if self.registration_year < 2018 and self.order_num >= 5:  
            self.discount = 0.1   
              
  
# Применение  
          
# Сделаем подобие базы  
client_db = [   
    Client("max@gmail.com", 2, 2019),  
    Client("lova@yandex.ru", 10, 2015),  
    Client("german@sberbank.ru", 4, 2017)  
]  
  
  
# Сгенерируем заказы  
client_db[0].make_order(100)  
client_db[1].make_order(200)  
client_db[2].make_order(500)  
client_db[2].make_order(500)  

Order price for max@gmail.com is 100
Order price for lova@yandex.ru is 180.0
Order price for german@sberbank.ru is 500
Order price for german@sberbank.ru is 450.0


In [2]:
# Классы могут пригодиться, если вы регулярно делаете над данными одну и ту же последовательность разноплановых 
# функций. Вы можете упаковать их в класс и в дальнейшем сразу получать результат по загруженным данным.

# У нас есть численные данные из разных источников. Если они в виде строк, то нужно привести их к числам, а 
# пропуски — заполнить значениями. Сделаем доступ к медиане, среднему значению и стандартному отклонению:

import statistics  
  
class DataFrame():  
    def __init__(self, column, fill_value=0):  
        # Инициализируем атрибуты  
        self.column = column  
        self.fill_value = fill_value  
        # Заполним пропуски  
        self.fill_missed()  
        # Конвертируем все элементы в числа  
        self.to_float()  
          
    def fill_missed(self):  
        for i, value in enumerate(self.column):  
            if value is None or value == '':  
                self.column[i] = self.fill_value  
                  
    def to_float(self):  
        self.column = [float(value) for value in self.column]  
      
    def median(self):  
        return statistics.median(self.column)  
      
    def mean(self):  
        return statistics.mean(self.column)  
      
    def deviation(self):  
        return statistics.stdev(self.column)  
      
  
      
# Воспользуемся классом  
df = DataFrame(["1", 17, 4, None, 8])  
  
print(df.column)  
print(df.deviation())   
print(df.median())  

[1.0, 17.0, 4.0, 0.0, 8.0]
6.892024376045111
4.0


In [3]:
# Определите класс IntDataFrame, который в момент инициализации объектов принимает список неотрицательных чисел и 
# приводит к целым значениям все числа в этом списке, отрезая дробную часть с помощью встроенной функции int().
# Результирующий список должен быть сохранен в виде атрибута с именем column.
# Также класс должен содержать следующие методы:
# count(), который возвращает количество ненулевых элементов в списке column;
# unique(), который возвращает число уникальных элементов в списке в списке column.

# Решение 1:
class IntDataFrame():
    def __init__(self, column):
        self.column = list(map(lambda x: int(x), column))       

    def count(self):
        return len(list(filter(lambda x: x!=0, self.column)))

    def unique(self):
        uniq = []
        for i, value in enumerate(self.column):
            if value in uniq:
                continue
            else:
                uniq.append(value)
        return len(uniq)

df = IntDataFrame([4.7, 4, 3, 0, 2.4, 0.3, 4])

print(df.column)
print(df.count())
print(df.unique())

[4, 4, 3, 0, 2, 0, 4]
5
4


In [4]:
# Решение 2 (моё):
import statistics
import pandas as pd
class IntDataFrame():
    def __init__(self, column):
        self.column = column
        self.int()
        
    def int(self):
        self.column = [int(value) for value in self.column]
        
    def count(self):
        count = 0
        for i, value in enumerate(self.column):  
            if self.column[i] > 0:  
                count += 1
        return count
    
    def unique(self):
        self.copy = pd.Series(self.column.copy())
        return self.copy.nunique(dropna=True)

df = IntDataFrame([4.7, 4, 3, 0, 2.4, 0.3, 4])

print(df.column)
print(df.count())
print(df.unique())

[4, 4, 3, 0, 2, 0, 4]
5
4


In [5]:
# Классы можно использовать тогда, когда у вас есть процесс, который требует сложной конфигурации, повторяющейся из 
# раза в раз. Можно написать класс-обёртку, который сведёт этот процесс к одному-двум методам.

# Представим, вы делаете обработку данных и в конце каждого дня сохраняете результат в архив. Вы хотите, чтобы данные 
# каждого дня лежали в отдельном файле для этого дня, при этом можно было бы получить данные за произвольный день. 
# Перед запуском кода создайте папку с названием archive там же, где находится ноутбук:

import pickle  
from datetime import datetime  
from os import path  
  
class Dumper():  
    def __init__(self, archive_dir="archive/"):  
        self.archive_dir = archive_dir  
          
    def dump(self, data):  
        # Библиотека pickle позволяет доставать и класть объекты в файл  
        with open(self.get_file_name(), 'wb') as file:  
            pickle.dump(data, file)  
              
    def load_for_day(self, day):  
        file_name = path.join(self.archive_dir, day + ".pkl")   
        with open(file_name, 'rb') as file:  
            sets = pickle.load(file)  
        return sets  
          
    # возвращает корректное имя для файла   
    def get_file_name(self):   
        today = datetime.now().strftime("%y-%m-%d")   
        return path.join(self.archive_dir, today + ".pkl")  
      
      
# Пример использования  
  
data = {  
    'perfomance': [10, 20, 10],  
    'clients': {"Romashka": 10, "Vector": 34}  
}  
  
  
dumper = Dumper()  
  
# Сохраним данные  
dumper.dump(data)  
  
# Восстановим для сегодняшней даты  
file_name = datetime.now().strftime("%y-%m-%d")
restored_data = dumper.load_for_day(file_name)
print(restored_data)  
# => {'perfomance': [10, 20, 10], 'clients': {'Romashka': 10, 'Vector': 34}} 

{'perfomance': [10, 20, 10], 'clients': {'Romashka': 10, 'Vector': 34}}


In [6]:
# Напишите класс сборщика технических сообщений OwnLogger.
# У него должен быть:
# атрибут logs, содержащий {"info": None, "warning": None, "error": None, "all": None}.
# метод log(message, level), который записывает сообщения. Здесь сообщение message может быть любым, а level — один 
# из "info", "warning", "error".
# метод show_last(level), где level может быть "info", "warning", "error", "all".
# Для "all" он просто возвращает последнее добавленное сообщение, а для остальных — последнее поступившее сообщение 
# соответствующего уровня. При этом по умолчанию значение именно "all".
# Если подходящего сообщения нет, возвращает None.    
        
class OwnLogger():
    def __init__(self):
        self.logs = {"info": None, "warning": None, "error": None, "all": None}
    def log(self, message, level):
        self.logs[level] = message
        self.logs['all'] = message
    def show_last(self, level='all'):
        return self.logs[level]

logger = OwnLogger()
logger.log("System started", "info")
logger.show_last("error")
# None
# Некоторые интерпретаторы Python могут не выводить None, тогда в этой проверке у вас будет пустая строка
logger.log("Connection instable", "warning")
logger.log("Connection lost", "error")

logger.show_last()
# Connection lost
logger.show_last("info")
# System started

'System started'

In [4]:
f = open('input.txt', 'w', encoding='utf8')

# Запишем в файл строку
f.write("This is a test string\n")
f.write("This is a new string\n") # 22
# обязательно нужно закрыть файл иначе он будет заблокирован ОС
f.close()

In [5]:
# Создайте любой файл на операционной системе под название input.txt и построчно перепишите его в файл output.txt.

with open("input.txt", "r") as input_file:
    with open("output.txt", "w") as output_file:
        for line in input_file:
            output_file.write(line)

In [7]:
# Дан файл numbers.txt, компоненты которого являются действительными числами (файл создайте самостоятельно и заполните 
# любыми числам, в одной строке одно число). Найдите сумму наибольшего и наименьшего из значений и запишите результат 
# в файл output.txt.

filename = 'numbers.txt'
output = 'output.txt'

with open(filename) as f:
    min_ = max_ = float(f.readline())  # считали первое число
    for line in f:
        num = float(line)
        if num > max_:
            max_ = num
        elif num < min_:
            min_ = num

    sum_ = min_ + max_

with open(output, 'w') as f:
    f.write(str(sum_))
    f.write('\n')

In [12]:
# В текстовый файл построчно записаны фамилии и имена учащихся класса и их оценки за контрольную. 
# Подсчитайте количество учащихся, чья оценка меньше 3 баллов

count = 0
for line in open("names.txt", encoding = 'utf8'):
    points = int(line.split()[-1])
    if points < 3:
        count += 1
count

4

In [13]:
# Выполните реверсирование строк файла (перестановку строк файла в обратном порядке).

with open("input.txt", "r") as input_file:
    with open("output.txt", "w") as output_file:
        for line in reversed(input_file.readlines()):
            output_file.write(line)